<a href="https://colab.research.google.com/github/chloesung/Econometrics/blob/main/2.%20Instrument%20Variables%20Approach/Wooldridge/Wooldridge_Ch15(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
pip install wooldridge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import re
import math

import seaborn as sns
sns.set_palette("pastel")
sns.set(style="whitegrid")
import warnings
import matplotlib.pyplot as plt
pal = sns.color_palette('Set1')

from scipy.stats import t
from statsmodels.formula.api import ols
pd.options.display.float_format = '{:.5f}'.format
plt.rcParams['axes.unicode_minus'] = False
import wooldridge

warnings.filterwarnings(action='ignore')
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina' #화질 좋게 해주기")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. College Proximity

## a) EDA

In [3]:
# Data Import
df = wooldridge.data('mroz')[['lwage', 'educ', 'fatheduc']].dropna()
df

,lwage,educ,fatheduc
0,1.21015,12,7
1,0.32851,12,7
2,1.51414,12,7
3,0.09212,12,7
4,1.52427,14,14
...,...,...,...
423,0.83803,10,7
424,1.66886,12,7
425,1.76943,13,7
426,1.22645,12,16


- $fatheduc$ is an instrument for education

## b) OLS


In [4]:
# Regression
res = ols('lwage ~  educ', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     56.93
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           2.76e-13
Time:                        07:27:09   Log-Likelihood:                -441.26
No. Observations:                 428   AIC:                             886.5
Df Residuals:                     426   BIC:                             894.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1852      0.185     -1.000      0.3

- $\beta_1$ implies an almost 11% return for another year of education.

## c) IV

In [5]:
# educ - fatheduc
res = ols('educ ~  fatheduc', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     88.84
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           2.76e-19
Time:                        07:27:09   Log-Likelihood:                -920.02
No. Observations:                 428   AIC:                             1844.
Df Residuals:                     426   BIC:                             1852.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.2371      0.276     37.099      0.0

- p-value is small enough to reject the null hypothesis, $\pi_1 = 0$.
- $educ$ and $fatheduc$ have a statistically significant positive correlation.

In [6]:
# 1st Stage
res = ols('educ ~ fatheduc', data=df).fit()
df['educ_iv'] = res.fittedvalues.values

# 2nd Stage
res = ols('lwage ~ educ_iv', data=df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.586
Date:                Thu, 18 Aug 2022   Prob (F-statistic):              0.109
Time:                        07:27:09   Log-Likelihood:                -466.81
No. Observations:                 428   AIC:                             937.6
Df Residuals:                     426   BIC:                             945.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4411      0.467      0.944      0.3

- $\beta_1$ implies an almost 6% return for another year of education.
- This *suggests* that the OLS estimate is too high and is consistent with omitted ability bias.
- However, although the differences are practically large, we cannot say whether the difference is statistically significant. 